# GLM dev
- "Kernel" GLM: Gaussian GLM with L2 regularization
- Kernel dropouts for "unique" contribution
- 4 different models
    - touch, whisker, touch-answer, whisker-answer
- Design matrix same as before
    - Try dividing answer lick onset, reward lick onset, and other licks
        - See how much better the model gets
- (consider) Multicolinearity test
- Perform per plane
- Refer Alex's code

In [4]:
import numpy as np
import pandas as pd
import xarray as xr
from pathlib import Path
import matplotlib.pyplot as plt

base_dir = Path(r'E:\TPM\JK\h5')

expert_mice_df = pd.read_csv(base_dir / 'expert_mice.csv', index_col=0)
use_mice_df = expert_mice_df.loc[expert_mice_df['depth_matched'].astype(bool) & 
                                 ~expert_mice_df['processing_error'].astype(bool) &
                                 ((expert_mice_df.session_type == 'training') |
                                  (expert_mice_df.session_type.str.contains('test')))]

# Design matrix dev
- using refined_frame_time.pkl
- parameters:
    - touch: 0-2 frames (2 angles)
    - whisking: -2-4 frames (number of whisks, whisking amplitude, midpoint)
        - These are not calculated yet. Add.
    - licking: -2-2 frames (left and right)
    - sound cue: 0-4 frames (single)
    - rewards: 0-4 frames (single)

In [12]:
mouse = 25
plane = 1
session = 3
plane_dir = base_dir / f'{mouse:03}/plane_{plane}'
behavior_frametime = pd.read_pickle(plane_dir / f'JK{mouse:03}_S{session:02}_plane{plane}_frame_whisker_behavior.pkl')
roi_dir = plane_dir / f'{session:03}/plane0/roi'
ophys_frametime = pd.read_pickle(roi_dir / 'refined_frame_time.pkl')


In [15]:
behavior_frametime.head()

,trialNum,frame_start_time,frame_end_time,trial_duration,frame_index,kappaH_onset,kappaV_onset,phi_onset,theta_onset,arc_length_onset,...,first_reward_lick_right,correct,wrong,miss,trial_type,task_target,distractor,mouse_name,session_name,session_type
0,2,0.221688,0.254117,6.101034,0,NaN,NaN,NaN,NaN,NaN,...,False,False,False,True,rn,Angle,Off,JK025,S03,2port-Discrim
1,2,0.351407,0.383836,6.101034,1,NaN,NaN,NaN,NaN,NaN,...,False,False,False,True,rn,Angle,Off,JK025,S03,2port-Discrim
2,2,0.481125,0.513555,6.101034,2,NaN,NaN,NaN,NaN,NaN,...,False,False,False,True,rn,Angle,Off,JK025,S03,2port-Discrim
3,2,0.610844,0.643274,6.101034,3,NaN,NaN,NaN,NaN,NaN,...,False,False,False,True,rn,Angle,Off,JK025,S03,2port-Discrim
4,2,0.740563,0.772993,6.101034,4,NaN,NaN,NaN,NaN,NaN,...,False,False,False,True,rn,Angle,Off,JK025,S03,2port-Discrim


In [16]:
behavior_frametime.keys()

Index(['trialNum', 'frame_start_time', 'frame_end_time', 'trial_duration',
       'frame_index', 'kappaH_onset', 'kappaV_onset', 'phi_onset',
       'theta_onset', 'arc_length_onset', 'touch_count', 'delta_kappaH',
       'delta_kappaV', 'delta_phi', 'delta_theta', 'slide_distance',
       'touch_duration', 'pole_up_frame', 'pole_moving_frame', 'pole_angle',
       'pole_ap_distance', 'pole_radial_distance', 'first_lick_frame',
       'first_lick_left', 'first_lick_right', 'num_lick_left',
       'num_lick_right', 'answer_lick_frame', 'answer_lick_left',
       'answer_lick_right', 'pre_answer_pole_up_frame', 'answer_period_frame',
       'post_answer_pole_up_frame', 'first_reward_lick_frame',
       'first_reward_lick_left', 'first_reward_lick_right', 'correct', 'wrong',
       'miss', 'trial_type', 'task_target', 'distractor', 'mouse_name',
       'session_name', 'session_type'],
      dtype='object')

In [14]:
ophys_frametime

,trialNum,frame_start_time,frame_end_time,trial_duration,frame_index,remove_frame,remove_trial
0,2,0.351407,0.383836,6.101034,1,False,False
1,2,0.481125,0.513555,6.101034,2,False,False
2,2,0.610844,0.643274,6.101034,3,False,False
3,2,0.740563,0.772993,6.101034,4,False,False
4,2,0.870282,0.902712,6.101034,5,False,False
...,...,...,...,...,...,...,...
7376,351,3.455730,3.488160,8.330198,7775,False,False
7377,351,3.585449,3.617879,8.330198,7776,False,False
7378,351,3.715168,3.747598,8.330198,7777,False,False
7379,351,3.844887,3.877317,8.330198,7778,False,False


### whisking data not included in the frametime pkl file
- It is in session whisker.pkl

In [17]:
w_dir = Path('E:\TPM\JK\WhiskerVideo')
session_name = f'JK{mouse:03}S{session:02}'
whisker_pkl_fn = w_dir / session_name / f'{session_name}_whisker.pkl'
whisker_df = pd.read_pickle(whisker_pkl_fn)

In [18]:
whisker_df.head()

,amplitude,arcLength,kappaH,kappaV,midpoint,phase,phi,poleAngle,poleDistance,poleMovingFrames,...,retractionSlideByWhisking,retractionTFchunks,retractionTFchunksByWhisking,retractionTouchDuration,retractionTouchDurationByWhisking,theta,whisker_time,whiskingStartFrames,mouse_name,session_name
trialNum,,,,,,,,,,,,,,,,,,,,,
2,"[0.1120494503411022, 0.0773610668341734, 0.047...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.028230989282557278, 0.027747604479175337, 0...","[-0.050709654585140436, -0.04912657045486456, ...","[-23.242985098222128, -23.243675680549472, -23...","[-1.428783523782121, -1.689815252745931, -2.37...","[3.6651240340198536, 3.687700830566926, 3.7135...",45,140000,"[420.0, 421.0, 422.0, 423.0, 424.0, 425.0, 426...",...,[],[],[],"[0, 0]","[0, 0]","[-23.182265522274143, -23.226476520847495, -23...","[0.0, 0.0032129546330805807, 0.006425909266161...","[154.0, 173.0, 198.0, 206.0, 234.0, 278.0, 309...",JK025,S03
3,"[2.663283131286702, 1.2759453714445574, 1.5299...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.02192368612008598, 0.022401161945715797, 0....","[-0.05148633621510797, -0.05179630738114133, -...","[-10.936527219253362, -11.003985094863797, -11...","[1.6497037362943963, 1.818804190358328, 1.8726...","[-4.10917501818044, -4.62078802516819, -4.3054...",45,140000,"[366.0, 367.0, 368.0, 369.0, 370.0, 371.0, 372...",...,"[[0.0, 0.37717595429835016, 0.5906804256218757...","[[1292.0, 1293.0, 1294.0, 1295.0, 1296.0], [13...","[[1397.0, 1398.0, 1399.0, 1400.0, 1401.0, 1402...","[0.016064773165402894, 0.009638863899241738, 0...","[0.019277727798483472, 0.009638863899241738]","[-11.094797307763693, -11.307534738373013, -11...","[0.0, 0.0032129546330805807, 0.006425909266161...","[1.0, 9.0, 173.0, 278.0, 292.0, 322.0, 375.0, ...",JK025,S03
4,"[1.9925797261745597, 1.7596101535969138, 1.520...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.020910858384018523, 0.022415314221942186, 0...","[-0.04987775821761756, -0.05243916130685084, -...","[-11.638586362860918, -11.668318024759381, -11...","[1.5389477210649638, 1.4768919925997976, 1.427...","[0.4164677051680596, 0.34778010269002274, 0.08...",135,140000,"[356.0, 357.0, 358.0, 359.0, 360.0, 361.0, 362...",...,[],[],[],"[0, 0]","[0, 0]","[-11.488843531003683, -11.488304016412187, -11...","[0.0, 0.0032129546330805807, 0.006425909266161...","[35.0, 87.0, 119.0, 146.0, 176.0, 183.0, 405.0...",JK025,S03
5,"[1.062988934657812, 0.8241170579025188, 0.8353...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.02050759312419506, 0.020693151389785915, 0....","[-0.05774239721185456, -0.05447276851004021, -...","[-10.980825014950286, -10.810623550762893, -10...","[2.3369688573159113, 2.5074619958064215, 2.388...","[5.5006593781319335, 5.363392207195308, 5.3302...",45,140000,"[366.0, 367.0, 368.0, 369.0, 370.0, 371.0, 372...",...,[],[],[],"[0, 0]","[0, 0]","[-11.899807322973114, -11.641251603733906, -11...","[0.0, 0.0032129546330805807, 0.006425909266161...","[1.0, 12.0, 72.0, 97.0, 152.0, 239.0, 472.0, 6...",JK025,S03
6,"[1.3215559856718835, 2.332453952677843, 2.7139...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.024081165203598157, 0.01909150455145891, 0....","[-0.05051955979391675, -0.052321209688238716, ...","[-6.202850280352959, -6.034399997117823, -5.86...","[1.4940845109864258, 2.1466234668591246, 2.659...","[13.44462735825601, 14.020582298270684, 13.949...",45,140000,"[366.0, 367.0, 368.0, 369.0, 370.0, 371.0, 372...",...,[],[],[],"[0, 0]","[0, 0]","[-6.1788849456395445, -7.794175390582096, -8.7...","[0.0, 0.0032129546330805807, 0.006425909266161...","[1.0, 5.0, 29.0, 48.0, 94.0, 132.0, 155.0, 181...",JK025,S03
